In [1]:
from bs4 import BeautifulSoup
from IPython.display import display, clear_output
from sklearn.preprocessing import PowerTransformer
import pandas as pd
import numpy as np
import requests
import talib
import json
import time
import io
import math
from datetime import datetime
import matplotlib.pyplot as plt

pt = PowerTransformer()

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 500)

In [2]:
header = {
          "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0",
          "X-Requested-With": "XMLHttpRequest"
        }

In [3]:
def getOneMinuteVolume(ticker):
    currentSymbol = (ticker + '.NS')
    period1 = int(time.time() - 1209600) #two calender week epoc
    period2 = int(time.time())
    url = f'https://query2.finance.yahoo.com/v8/finance/chart/{currentSymbol}?formatted=true&crumb=LqSYCwMewDq&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&events=capitalGain|div|split&useYfid=true&corsDomain=finance.yahoo.com'
    currentStockUrl = f'https://query1.finance.yahoo.com/v7/finance/download/{currentSymbol}?period1={period1}&period2={period2}&interval=1d&events=history&includeAdjustedClose=false'
    try:
        r = requests.get(url, headers=header)
        soup = json.loads(str(BeautifulSoup(r.text, 'html.parser')))
        oneMinuteVolume = np.mean(soup['chart']['result'][0]['indicators']['quote'][0]['volume'][-6:-1])/385
    except:
        tempDf = pd.read_csv(currentStockUrl)
        oneMinuteVolume = tempDf[-6:-1]['Volume'].median()/385
    print(ticker, oneMinuteVolume)
    return oneMinuteVolume

In [4]:
valuePage = f'https://etmarketsapis.indiatimes.com/ET_Stats/moversvalue?pagesize=500&exchange=nse&pageno=1&service=valuemovers&sortby=value&sortorder=desc'
gainersPage = f'https://etmarketsapis.indiatimes.com/ET_Stats/moversvalue?pagesize=1000&exchange=nse&pageno=1&service=valuemovers&sortby=percentchange&sortorder=desc'
losersPage = f'https://etmarketsapis.indiatimes.com/ET_Stats/moversvalue?pagesize=1000&exchange=nse&pageno=1&service=valuemovers&sortby=percentchange&sortorder=asc'

In [5]:
def getPriceChange(priceSeries):
    if len(priceSeries) > 15:
        priceSeries = priceSeries[-15:]
        try:
            priceChange = np.log(priceSeries[-1]/priceSeries[0])
        except Exception as e:
            print(e, '2')
            priceChange = np.nan
    else:
        print(len(priceSeries))
        priceChange = np.nan
    return priceChange

def getVolumeChange(volumeSeries):
    if len(volumeSeries) > 5:
        volumeSeries = volumeSeries[-5:]
        try:
            volChange = np.log(volumeSeries[-1]/volumeSeries[0])
        except Exception as e:
            print(e, '3')
            volChange = np.nan
    else:
        print(len(volumeSeries))
        volChange = np.nan
    return volChange

In [6]:
colNames = ['companyName', 'Ticker', 'Price', 'Change', 'Value','volumeMultiple', 'tradeQuantity','priceChange', 'volChange', 'compositeRating', 'Time']
oneMinuteVolumeDict = dict()
priceDict = dict()
volumeDict = dict()
signalDataframe = pd.DataFrame(columns=colNames)

In [7]:
%%time
while True:
    try:
        while True:
            try:
                valuePageRequest = requests.get(valuePage, headers = header, timeout = 10)
            except:
                print(valuePageRequest.status_code, ' Request Failed')
            if valuePageRequest.status_code == 200:
                break
        while True:
            try:
                gainersPageRequest = requests.get(gainersPage, headers = header, timeout = 10)
            except:
                print(gainersPageRequest.status_code, ' Request Failed')
            if gainersPageRequest.status_code == 200:
                break
        while True:
            try:
                losersPageRequest = requests.get(losersPage, headers = header, timeout = 10)
            except:
                print(losersPageRequest.status_code, ' Request Failed')
            if losersPageRequest.status_code == 200:
                break

        valuePageDict = json.loads(str(BeautifulSoup(valuePageRequest.text, 'html.parser')))
        gainersPageDict = json.loads(str(BeautifulSoup(gainersPageRequest.text, 'html.parser')))
        losersPageDict = json.loads(str(BeautifulSoup(losersPageRequest.text, 'html.parser')))

        valuePageSet = set(symbol['ticker'] for symbol in valuePageDict['searchresult'])
        gainersPageeSet = set(symbol['ticker'] for symbol in gainersPageDict['searchresult'])
        losersPageSet = set(symbol['ticker'] for symbol in losersPageDict['searchresult'])

        valueGainers = set.intersection(valuePageSet, gainersPageeSet)
        valueLosers = set.intersection(valuePageSet, losersPageSet)

        screenerStockSymbols = valuePageSet
        screenerStockSymbols = set.union(valueGainers, valueLosers)

        volumeMultiple = list()
        tradeQuantity = list()

        screenerDataDictList = list()

        for symbolData in valuePageDict['searchresult']:
            try:
                #Filters
                if symbolData['ticker'] in screenerStockSymbols:

                    #Appending price
                    if symbolData['ticker'] in priceDict.keys():
                        priceDict[symbolData['ticker']].append(symbolData['current'])
                    else:
                        priceDict[symbolData['ticker']] = list()
                        priceDict[symbolData['ticker']].append(symbolData['current'])

                    #Appending volume
                    if symbolData['ticker'] in volumeDict.keys():
                        volumeDict[symbolData['ticker']].append(symbolData['volume'])
                    else:
                        volumeDict[symbolData['ticker']] = list()
                        volumeDict[symbolData['ticker']].append(symbolData['volume'])

                    lastClose = symbolData['current'] - symbolData['absoluteChange']
                    openPrice = symbolData['open']
                    currentPrice = symbolData['current']
                    currentVolume = symbolData['volume']
                    minuteSinceOpen = (datetime.strptime((str(datetime.now())[11:-10]), '%H:%M') - datetime.strptime(('09:07'), '%H:%M')).total_seconds()/60
                    try:
                        weekHighBroken = symbolData['high'] == symbolData['weekHighPrice']
                        weekLowBroken = symbolData['low'] == symbolData['weekLowPrice']
                    except:
                        weekHighBroken = True
                        weekLowBroken = True
                    
                    #For Research:
                    if (minuteSinceOpen > 385) or (minuteSinceOpen < 0):
                        minuteSinceOpen = 385

                    if symbolData['ticker'] in oneMinuteVolumeDict.keys():
                        adjustedWeekVolume = oneMinuteVolumeDict[symbolData['ticker']] * minuteSinceOpen
                    else:
                        try:
                            oneMinuteVolumeDict[symbolData['ticker']] = getOneMinuteVolume(symbolData['ticker'])
                            adjustedWeekVolume = oneMinuteVolumeDict[symbolData['ticker']] * minuteSinceOpen
                        except Exception as e:
                            print(e)
                            print('DEAD!!! ', symbolData['ticker'])
                            pass
                    #Check Gap Up/Down
                    #Append Data
                    screenerDataDictList.append(symbolData)
                    #Additional Features
                    volumeMultiple.append(currentVolume/(adjustedWeekVolume + 1))
                    tradeQuantity.append(int((300 * 74.58)/symbolData['current']))
            except Exception as er:
                    print(er, '5')
                    continue
                    
        screenerDataframe = pd.DataFrame(screenerDataDictList)[['companyName', 'ticker', 'current', 'percentChange', 'value']]
        screenerDataframe = screenerDataframe.join(pd.DataFrame.from_dict({'volumeMultiple' : volumeMultiple}))
        screenerDataframe = screenerDataframe.join(pd.DataFrame.from_dict({'tradeQuantity' : tradeQuantity}))
        screenerDataframe.columns = ['companyName', 'Ticker', 'Price', 'Change', 'Value', 'volumeMultiple', 'tradeQuantity']

        #Value Filters
        screenerDataframe = screenerDataframe[screenerDataframe['Value'].astype('float') > float(valuePageDict['searchresult'][450]['value'])]
        
        #Price and Volume Change Column
        priceChangeRatioCol = list()
        volChangeCol = list()
        for ticker in screenerDataframe['Ticker'].values:
            priceChangeRatioCol.append(abs(getPriceChange(priceDict[ticker])))
            volChangeCol.append(getVolumeChange(volumeDict[ticker]))

        screenerDataframe['priceChange'] = priceChangeRatioCol
        screenerDataframe['volChange'] = volChangeCol

        #Composite Compare
        screenerDataframe['volChange'] = pt.fit_transform(np.array(screenerDataframe['volChange']).reshape(-1,1))
        screenerDataframe['priceChange'] = pt.fit_transform(np.array(screenerDataframe['priceChange']).reshape(-1,1))
        
        screenerDataframe['volChange'] = screenerDataframe['volChange'] #* -1 #Price not supported by Vol
        
        screenerDataframe['compositeRating'] = (screenerDataframe['volChange'] + screenerDataframe['priceChange'])/2

        screenerDataframe = screenerDataframe.sort_values(by = ['compositeRating'], ascending = False, ignore_index = True)
        
        #Signal DataFrame
        signalDataframeTickerSet = set(signalDataframe[:]['Ticker'])
        screenerDataframeTickerSet = set(screenerDataframe[screenerDataframe['compositeRating'] > 1.9]['Ticker'])
        newSignal = screenerDataframeTickerSet - signalDataframeTickerSet
        for newSig in newSignal:
            tempSignalDataFrame = screenerDataframe[screenerDataframe['Ticker'] == newSig]
            tempSignalDataFrame['Time'] = str(datetime.now())[11:-10]
            signalDataframe = signalDataframe.append(tempSignalDataFrame)

        clear_output(wait=True)
        display(screenerDataframe)
        display(signalDataframe.reset_index())
        print(str(datetime.now())[:-10], " Minimum Volume in Cr : ", valuePageDict['searchresult'][150]['value'])
        print("Minutes Since Open : ", minuteSinceOpen)
        time.sleep(55)
    except Exception as e:
        print(e, '6')
        time.sleep(2)

,companyName,Ticker,Price,Change,Value,volumeMultiple,tradeQuantity,priceChange,volChange,compositeRating
0,Indian Raily Catering And Tourism,IRCTC,5463.55,10.85,7663.28,2.127518,4,NaN,NaN,NaN
1,Tata Motors,TATAMOTORS,497.60,-1.83,5211.18,0.778396,44,NaN,NaN,NaN
2,Wipro,WIPRO,708.25,5.30,4244.19,6.759035,31,NaN,NaN,NaN
3,Mindtree,MINDTREE,4691.20,7.50,3612.46,5.883643,4,NaN,NaN,NaN
4,Infosys,INFY,1715.75,0.38,3416.39,2.925089,13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
445,DCAL,DCAL,227.95,-3.96,16.47,0.494309,98,NaN,NaN,NaN
446,Vardhman Textiles,VTL,2033.05,-0.37,16.41,0.342349,11,NaN,NaN,NaN
447,Mishra Dhatu Nigam,MIDHANI,200.20,0.98,16.33,0.571270,111,NaN,NaN,NaN
448,Tata Metaliks,TATAMETALI,1076.60,-1.04,16.31,0.427398,20,NaN,NaN,NaN


,index,companyName,Ticker,Price,Change,Value,volumeMultiple,tradeQuantity,priceChange,volChange,compositeRating,Time


2021-10-17 16:17  Minimum Volume in Cr :  97.98
Minutes Since Open :  385


KeyboardInterrupt: 

In [8]:
screenerDataframe.sort_values(by = ['priceChange'], ascending = False, ignore_index = True)

,companyName,Ticker,Price,Change,Value,volumeMultiple,tradeQuantity,priceChange,volChange,compositeRating
0,Indian Raily Catering And Tourism,IRCTC,5463.55,10.85,7663.28,2.127518,4,NaN,NaN,NaN
1,Tata Motors,TATAMOTORS,497.60,-1.83,5211.18,0.778396,44,NaN,NaN,NaN
2,Wipro,WIPRO,708.25,5.30,4244.19,6.759035,31,NaN,NaN,NaN
3,Mindtree,MINDTREE,4691.20,7.50,3612.46,5.883643,4,NaN,NaN,NaN
4,Infosys,INFY,1715.75,0.38,3416.39,2.925089,13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
445,DCAL,DCAL,227.95,-3.96,16.47,0.494309,98,NaN,NaN,NaN
446,Vardhman Textiles,VTL,2033.05,-0.37,16.41,0.342349,11,NaN,NaN,NaN
447,Mishra Dhatu Nigam,MIDHANI,200.20,0.98,16.33,0.571270,111,NaN,NaN,NaN
448,Tata Metaliks,TATAMETALI,1076.60,-1.04,16.31,0.427398,20,NaN,NaN,NaN
